# BTM Data science notebook

In this session we will use examples from https://github.com/pycam/python-data-science condensed down to a 1 and a half hour session, to get familiar with pandas and classes. 


## Pandas 
Pandas is a data analysis module that allows you to use python with tables and data objects, much like data.frames in R.

Lets get started by opening a table and look at some data using pandas. We will also look at how to do the same tasks in python and see how much simpler pandas can be sometimes.

### Reading a table without pandas 

In [ ]:
import csv
data = []
with open("data/genes.txt") as f:
    reader = csv.DictReader(f, delimiter = "\t")
    for row in reader:
        data.append(row)
# look at the data
for d in data:
    print(d['gene'], d['chrom'], d['start'], d['end'])

### Loading the same data using pandas

In [ ]:
import pandas as pd
df = pd.read_csv("data/genes.txt", sep = "\t")
df.head()

As you can see, using pandas you can be quicker in writing code that handles data tables, but you also do not have total control over what happens, so sometimes using pandas might not what you need. This you will have to decide every times new.

### Exploring the data
When you firts open a data file in jupyter lab, you might want to check that the data was correctly loaded. A good way to do this is to actually lookt at the data. Pandas allows you to do this. Lets check it out.

In [ ]:
# load pandas, although it should already be loaded into this kernel
import pandas as pd
# open a dataset
df = pd.read_csv('data/GRCm38.tsv', sep='\t')

In [ ]:
# lets look at the first few rows
df.head()

In [ ]:
# lets check out the column types
df.dtypes

In [ ]:
# how large is the table (rows, columns)
df.shape

In [ ]:
# what are the column names
df.columns.values

In [ ]:
# for numeric data pandas can give you some stats, so you can check for basic errors
df.describe()

In [ ]:
# sorting the table based on column
df = df.sort_values(['start', 'end', 'strand'])
df.head()

In [ ]:
# get the first two rows
df[:2]

In [ ]:
# get only the first two seqids
df[:2]['seqid']

In [ ]:
# filter by value and show top hits
df[df.type == "CDS"].head()

### selecting using iloc and loc
We see that `.iloc[0]` will return the first row in the DataFrame, while `.loc[0]` will return the row with index 0:

In [ ]:
df.iloc[0]

In [ ]:
print(df.head())

In [ ]:
df.loc[0]

### To summarise...

In [ ]:
# Selecting a column by name
df['seqid']

In [ ]:
# Selecting columns by name
df[['seqid', 'source']]

In [ ]:
# Selecting rows
df[0:3]

In [ ]:
# Selecting by position - on both axes
df.iloc[1:3, 0:2]

In [ ]:
# Selecting by label - on both axes
df.loc[[1, 2, 3], ['seqid', 'source']]

### Some quick exercises

- How many rows have a source of 'Gnomon'?
- Which seqid has the largest value in the 'end' column?
- What are all the unique values in the 'type' column?

# Functions and Classes
When we write code it is good practice to use functions and classes. They provide structure for your code and make it not only understandable but also reusable. 

## Functions
Functions are short generalized code structures that can be called with an input then do some computation and return an output. 


In [ ]:
def this_is_the_function_name(input_argument1, input_argument2):

    # The body of the function is indented
    # This function prints the two arguments to screen
    print('The function arguments are:', input_argument1, "and", input_argument2, '\n(this is done inside the function!)')

    # And returns their product
    return input_argument1 * input_argument2

In [ ]:
# call the function
r = this_is_the_function_name(100, 2)
print(f"The result of the computation is: {r}")

A basic example...

In [ ]:
def square (x):
    result = x*x
    
    return result

In [ ]:
print(square(2))
print(square(4))

## Classes
Classes are larger structures that are defined to group functions around a root case together. Commonly they provide functions for a certain data object. So you could for example imagine a tsv class, which provides functionality like loading, sorting and filtering of a tsv file. Here is an example how this could look like:

In [ ]:
class tsvClass():
    def __init__(self, path, autoload = False):
        '''
        When initialized the class will always execute this __init__ function.
        So whatever we do here, will always be done.
        '''
        self.path = path
        if autoload:
            self.load()
    
    def load(self):
        '''
        In this function we load the tsv file by using pandas
        '''
        self.df = pd.read_csv(self.path, sep = '\t')
    
    def filter(self, column = "type", match = "CDS"):
        '''
        This filter function takes two arguments and returns a subset
        based on these
        '''
        return self.df[self.df[column] == match]
        

In [ ]:
GRCm38 = tsvClass('data/GRCm38.tsv')
GRCm38.load()
GRCm38.df.head()

In [ ]:
# now we can use our filter function to reduce to exons
GRCm38.filter(match = "exon").head()

In [ ]:
# or we filter on the strand
GRCm38.filter(column = "strand", match = "+").head()

And because this is a class we now can go back to the tsv object and just ask it what file it was from

In [ ]:
GRCm38.path

I hope you now got a glimpse in how to write functions, you will need time and practize to write classes that are not just spagetti code in disguise but it is worth it.

Often a script can be split into functions and then put into a class which makes it more understandable and you will find bugs much quicker.


# Project 
We want to query ensembl with coding regions from different species. For this in the data folder you find the gff files of mouse, human and zebra fish. We will load the gff3 files and sample 100 coding sequences (CDS) for each and compare GC content between species by plotting the result. 

You find all needed code here you just need to make it work. If you have the time, make it nice.

In [ ]:
import requests, sys
import random
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
def get_genomic_content(chrom, start, end, species = 'human'):
    server = "https://rest.ensembl.org"
    ext = f"/sequence/region/{species}/{chrom}:{start}..{end}:1?"

    r = requests.get(server+ext, headers={ "Content-Type" : "text/plain"})

    if not r.ok:
      r.raise_for_status()

    return r.text

def calculateGC(seq):
    '''
    takes a genomic sequence and only counts AGCT items and returns simple GC content
    Ambigious bases are ignored
    '''
    seq = seq.lower()
    d = {}
    for c in "agct":
        d[c] = seq.count(c)
    gc = (d['g'] + d['c'])/(d['a'] + d['g'] + d['c'] + d['t'])
    return gc

In [ ]:
class gff3():
    '''
    This is a custom class for parsing esembles gff3 format
    Usually you might want to resort to known established parsers such as 
    provided by biopython: https://biopython.org/wiki/GFF_Parsing
    But their gff parser is not yet done. 
    For fasta and other file formats you can always rely on them though.
    '''
    def __init__(self, path, autoload = True):
        '''
        When initialized the class will always execute this __init__ function.
        So whatever we do here, will always be done.
        '''
        self.path = path
        if autoload:
            self.load()
    
    def load(self):
        '''
        In this function we load the gff3 file by using pandas
        '''
        knownnames = ['seqid', 'source', 'type',
                      'start', 'end', 'score', 
                      'strand', 'phase', 'attributes']
        self.df = pd.read_csv(self.path, sep = '\t', comment="#", names = knownnames)
    
    def filter(self, column = "type", match = "CDS"):
        '''
        This filter function takes two arguments and returns a subset
        based on these
        '''
        return self.df[self.df[column] == match]
    

In [ ]:
# load the mouse gff3 file
c = gff3('data/Mus_musculus.GRCm38.98.gff3.gz')
# then filter for CDS regions
df = c.filter()

In [ ]:
# initialize an empty list to store the results
res = []
# iterate the dataframe after sampleing
for index, row in df.sample(100).iterrows():
    # fetch the sequence using the esemble API
    seq = get_genomic_content(row.seqid, row.start, row.end, species = 'mouse')
    # calculate the GC content
    gc = calculateGC(seq)
    # create a dictionary for each result
    result = {'species': 'mouse',
              'gc': gc}
    # and save this in out result list
    res.append(result)

# transform the dictionary into a data frame
result_df = pd.DataFrame(res)
    

In [ ]:
# plot simple historgram
result_df.hist()

Now that you saw how to do it for one species, you can do the same for human and zebrafish and plot them together. You can merge pandas dataframes using `pd.concat`. Check the documentation for help. 

In [ ]:
files = ['Danio_rerio.GRCz11.98.gff3.gz', 'Homo_sapiens.GRCh38.98.gff3.gz',  'Mus_musculus.GRCm38.98.gff3.gz']
species =  ['Zebrafish', 'Human', 'Mouse']

In [ ]:
print("only scroll down If you want to see my solution. Else work in here")

In [ ]:
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#
#


In [ ]:
## Solution
class gffsample(gff3):
    '''
    this class inherits the functionality of the gff class and 
    expands it with new functionality for sampeling. But relies on outside functions
    for computing gc and fetching the data.
    Would be nice to include these into this class.
    '''
    def __init__(self, path, species = "human"):
        '''
        When initialized the class will always execute this __init__ function.
        So whatever we do here, will always be done.
        '''
        self.path = path
        # filter
        self.load()
        # also filter this time
        self.df = self.filter()
        # set the species
        self.species = species
        self.df_gc_created = False
    
    def sample(self, n = 100):
        self.sampled = self.df.sample(n)
    
    def make_GC(self):
        res = []
        for index, row in self.sampled.iterrows():
            # fetch the sequence using the esemble API
            seq = get_genomic_content(row.seqid, row.start, row.end, species = self.species)
            # calculate the GC content
            gc = calculateGC(seq)
            # create a dictionary for each result
            result = {'species': self.species,
                      'gc': gc}
            # and save this in out result list
            res.append(result)
        # transform the dictionary into a data frame
        self.df_gc = pd.DataFrame(res)
    
    def get_GC(self):
        # if not done already compute the GC content by querying the esemble api
        if self.df_gc_created == False:
            self.make_GC()
            self.df_gc_created = True
        
        return self.df_gc
        

In [ ]:
dataframes = []
for file, specie in zip(files, species):
    print(f'Sampling file: {file} ({specie})')
    g = gffsample(f'data/{file}', species = specie)
    g.sample(100)
    dataframes.append(g.get_GC())

In [ ]:
pd.concat(dataframes).hist(by = "species")

### Modules

Modules are a great way to structure / share your code to make it easily re-usable. Modules are simply python scripts (.py)

In [ ]:
import example_module as exp

res = []
# iterate the dataframe after sampling
for index, row in df.sample(3).iterrows():
    # fetch the sequence using the ensembl API
    seq = exp.get_genomic_content(row.seqid, row.start, row.end, species = 'mouse')
    # calculate the GC content
    gc = exp.calculateGC(seq)
    # create a dictionary for each result
    result = {'species': 'mouse',
              'gc': gc}
    # and save this in out result list
    res.append(result)
    
print(res)

Packages like Pandas etc... are simply collections of modules.

### Managing python environments

Environments are a way to manage which version of Python / packages you use. Setting up multiple environments, can allow you to quickly switch between different package versions...

The most popular ways to do this are with 'virtualenv' and 'conda'. Here we cover using conda:

By default you will have a base environment with all your current packages. You can list available functions by typing the following command in the terminal:

You can make a new environment like so:

To use a new environment we must 'activate' it:

We can also list the packages in our current environment:

We can save our environments as .yml files, that others can then use to create the exact same environment

To create an environment from a .yml file:

https://docs.conda.io/projects/conda/en/latest/user-guide/tasks/manage-environments.html#id3

### More info?

- The full Cambridge course: http://pycam.github.io/ 
- Bio-IT courses: https://bio-it.embl.de/
- EMBL chat - python & R channels
- EPUG & emblr
- Useful packages: https://www.scipy.org/